In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import pandas as pd
import numpy as np
from collections import Counter
import seaborn as sns

Duplicate key in file WindowsPath('c:/Users/USER/anaconda3/envs/python3.9.15/lib/site-packages/matplotlib/mpl-data/matplotlibrc'), line 273 ('font.monospace:  DejaVu Sans Mono, Bitstream Vera Sans Mono, Computer Modern Typewriter, Andale Mono, Nimbus Mono L, Courier New, Courier, Fixed, Terminal, monospace')


In [2]:
course_chapter_items_df = pd.read_csv('./data/course_chapter_items.csv')
course_df = pd.read_csv('./data/courses.csv')
users_df = pd.read_csv('./data/users.csv')
subgroups_df = pd.read_csv('./data/subgroups.csv')

train_group_df = pd.read_csv('./data/train_group.csv')
test_seen_group_df = pd.read_csv('./data/test_seen_group.csv')
val_seen_group_df = pd.read_csv('./data/val_seen_group.csv')
test_unseen_group_df = pd.read_csv('./data/test_unseen_group.csv')
val_unseen_group_df = pd.read_csv('./data/val_unseen_group.csv')

train_df = pd.read_csv('./data/train.csv')
test_seen_df = pd.read_csv('./data/test_seen.csv')
val_seen_df = pd.read_csv('./data/val_seen.csv')
test_unseen_df = pd.read_csv('./data/test_unseen.csv')
val_unseen_df = pd.read_csv('./data/val_unseen.csv')

In [3]:
def check_row_number_with_unique_amount(df, df_name, column):
    nrow = df.shape[0]
    nunique = pd.unique(df.loc[:,column]).shape[0]
    msg = f'dataframe {df_name} row number != unique {column} amount'
    assert nrow == nunique, msg
    print(f'dataframe {df_name} row number == unique {column} amount ({nrow} == {nunique})')

def check_unique_items_are_same_set(df1, df1_name, df1_column, df2, df2_name, df2_column):
    df1_set = set(df1.loc[:,df1_column].to_list())
    df2_set = set(df2.loc[:,df2_column].to_list())
    diff = df1_set.symmetric_difference(df2_set)
    msg = f'unique {df1_name}.{df1_column} & unique {df2_name}.{df2_column} are different'
    assert len(diff) == 0, msg
    print(f'unique {df1_name}.{df1_column} & unique {df2_name}.{df2_column} are the same')


# Use LightFM model dataset

## Build json file in format("user_id", "course_id")

In [4]:
users_df.info()
train_df.info()
val_seen_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130566 entries, 0 to 130565
Data columns (total 5 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   user_id            130566 non-null  object
 1   gender             85371 non-null   object
 2   occupation_titles  29056 non-null   object
 3   interests          82756 non-null   object
 4   recreation_names   31935 non-null   object
dtypes: object(5)
memory usage: 5.0+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59737 entries, 0 to 59736
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    59737 non-null  object
 1   course_id  59737 non-null  object
dtypes: object(2)
memory usage: 933.5+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7748 entries, 0 to 7747
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    7748 non

### Build Train Data

In [5]:
train_data_df = train_df
train_data_df['course_id'] = train_df['course_id'].apply(lambda x: [i for i in x.split(' ')])
train_data_df.set_index('user_id')['course_id'].apply(pd.Series).reset_index()

data_train_split = pd.melt(train_data_df.set_index('user_id')['course_id'].apply(pd.Series).reset_index(), 
             id_vars=['user_id'],
             value_name='course_id') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['user_id', 'course_id']) \
    .agg({'course_id': 'count'}) \
    .rename(columns={'course_id': 'have_purchased'}) \
    .reset_index() \
    .rename(columns={'course_id': 'course_id'})

data_train_split

,user_id,course_id,have_purchased
0,54dca4456d7d350900e86bae,56f521864e3ef90900b82c12,1
1,54dca4456d7d350900e86bae,601d03e07e6747ef11bba066,1
2,54e421bac5c9c00900cd8d47,6030c9cd99e14cc2401e66b9,1
3,54e961d4c5c9c00900cd8d84,575045e7206cd60900c53085,1
4,54e961d4c5c9c00900cd8d84,6014e3a6910e49742061f87e,1
...,...,...,...
139603,62bd5edfc2d8b50006215c3c,60f83c44b355920006abbd3f,1
139604,62e09de8fc3d3500060d4211,5f7c209762ad22756c7a1c74,1
139605,62e09de8fc3d3500060d4211,5f7c210b1de7982fb413a3e9,1
139606,62e09de8fc3d3500060d4211,5f7c212262ad2203e77a1cc9,1


### Build Valid Data

In [6]:
val_seen_data_df = pd.read_csv('./data/val_seen.csv')
val_seen_data_df['course_id'] = val_seen_df['course_id'].apply(lambda x: [i for i in x.split(' ')])
val_seen_data_df.set_index('user_id')['course_id'].apply(pd.Series).reset_index()

data_val_seen_split = pd.melt(val_seen_data_df.set_index('user_id')['course_id'].apply(pd.Series).reset_index(), 
             id_vars=['user_id'],
             value_name='course_id') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['user_id', 'course_id']) \
    .agg({'course_id': 'count'}) \
    .rename(columns={'course_id': 'have_purchased'}) \
    .reset_index() \
    .rename(columns={'course_id': 'course_id'})

data_val_seen_split

,user_id,course_id,have_purchased
0,54e961d4c5c9c00900cd8d84,5b3b080baa5ff8001e22316e,1
1,54e961d4c5c9c00900cd8d84,5e2a980ee3832b00247239d0,1
2,54e961d4c5c9c00900cd8d84,60dd6d80638ed00aa99b5556,1
3,54ef34e2b839040f00016086,6156a77fdf426a0007cc5fe1,1
4,54ef3574b839040f00016090,5d42554a2fb5020020621025,1
...,...,...,...
17812,614d65d5e7a5df000756b5a4,5a0be0f7a15b3f001e9842ec,1
17813,614d65d5e7a5df000756b5a4,5ff2a7d52a0eb0da5d2c898d,1
17814,615982c5a40fbc0006377d7a,5e46b7f85b02f600245ade9b,1
17815,618b59cd152ad900060f4441,60dd6d80638ed00aa99b5556,1


In [7]:
data_train_val_df = pd.concat([data_train_split, data_val_seen_split], ignore_index=True)
data_train_val_df

,user_id,course_id,have_purchased
0,54dca4456d7d350900e86bae,56f521864e3ef90900b82c12,1
1,54dca4456d7d350900e86bae,601d03e07e6747ef11bba066,1
2,54e421bac5c9c00900cd8d47,6030c9cd99e14cc2401e66b9,1
3,54e961d4c5c9c00900cd8d84,575045e7206cd60900c53085,1
4,54e961d4c5c9c00900cd8d84,6014e3a6910e49742061f87e,1
...,...,...,...
157420,614d65d5e7a5df000756b5a4,5a0be0f7a15b3f001e9842ec,1
157421,614d65d5e7a5df000756b5a4,5ff2a7d52a0eb0da5d2c898d,1
157422,615982c5a40fbc0006377d7a,5e46b7f85b02f600245ade9b,1
157423,618b59cd152ad900060f4441,60dd6d80638ed00aa99b5556,1


## Build dataset and Mapping

In [8]:
import json
from lightfm.data import Dataset

dataset = Dataset()
# use train and valid item ids
dataset.fit((x for x in data_train_val_df['user_id']),
            (x for x in data_train_val_df['course_id']))


c:\Users\USER\anaconda3\envs\python3.9.15\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [9]:
user_id_mapping, user_feature_mapping, item_id_mapping, item_feature_mapping = dataset.mapping()
id_user_mapping = {v : k for k, v in user_id_mapping.items()}
id_item_mapping = {v : k for k, v in item_id_mapping.items()}

In [10]:
num_users, num_items = dataset.interactions_shape()
print('Num users: {}, num_items {}.'.format(num_users, num_items))

Num users: 59737, num_items 698.


In [11]:
(training_data, weights_train) = dataset.build_interactions(((x, y) for x, y in zip(data_train_split['user_id'], data_train_split['course_id'])))
(valid_data, weights_valid) = dataset.build_interactions(((x, y) for x, y in zip(data_val_seen_split['user_id'], data_val_seen_split['course_id'])))
(valid_data_data, weights_all) = dataset.build_interactions(((x, y) for x, y in zip(data_train_val_df['user_id'], data_train_val_df['course_id'])))
print(repr(training_data))
print(repr(valid_data))
print(repr(valid_data_data))

<59737x698 sparse matrix of type '<class 'numpy.int32'>'
	with 139608 stored elements in COOrdinate format>
<59737x698 sparse matrix of type '<class 'numpy.int32'>'
	with 17817 stored elements in COOrdinate format>
<59737x698 sparse matrix of type '<class 'numpy.int32'>'
	with 157425 stored elements in COOrdinate format>


In [12]:
user_item_matrix = weights_train.toarray()
valid_user_item_matrix = weights_valid.toarray()
all_user_item_matrix = weights_all.toarray()
# for i in range(user_item_matrix.shape[0]):
#     for j in range(user_item_matrix.shape[1]):
#         if user_item_matrix[i][j]:
#             print(f"{id_user_mapping[i]}, {id_item_mapping[j]}")

## Train

In [51]:
from lightfm import LightFM

model = LightFM(loss='warp', learning_rate=0.01)
model.fit(training_data, epochs=100, num_threads=4, verbose=True)

Epoch: 100%|██████████| 10/10 [00:02<00:00,  3.74it/s]


## Evaluate

In [52]:
from lightfm.evaluation import precision_at_k
from lightfm.evaluation import auc_score


train_precision = precision_at_k(model, training_data, k=10).mean()
test_precision = precision_at_k(model, valid_data, k=10).mean()

train_auc = auc_score(model, training_data).mean()
test_auc = auc_score(model, valid_data).mean()

print('Precision: train %.5f, test %.5f.' % (train_precision, test_precision))
print('AUC: train %.5f, test %.5f.' % (train_auc, test_auc))

Precision: train 0.16198, test 0.02765.
AUC: train 0.97910, test 0.53943.


# Prediction

In [45]:
#test_seen_df = pd.read_csv('./data/val_seen.csv')
test_seen_df = pd.read_csv('./data/test_seen.csv')
test_seen_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7205 entries, 0 to 7204
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   user_id    7205 non-null   object
 1   course_id  7205 non-null   object
dtypes: object(2)
memory usage: 112.7+ KB


In [46]:
test_id = test_seen_df["user_id"].to_list()
test_course = test_seen_df["course_id"].to_list()
test_seen_info = {}
for id, course in zip(test_id, test_course):
    test_seen_info[id] = course


In [47]:
def sample_recommendation(model, user_ids, user_id, top = 5):
     
    recommend_list = []
    recommend_list_items_id_list = []
    for user_id in user_ids:
        user_id_index = user_id_mapping[user_id]
        scores = model.predict(user_id_index, np.arange(698))   # 698個subgroup
    

        top_items_index = np.argsort(-scores).tolist()
        top_items = []
        for item_index in top_items_index:
            if len(top_items) < top:
                if all_user_item_matrix[user_id_index][item_index] != 1:
                    top_items.append(item_index)
                    
        recommend_list.append([id_item_mapping[x] for x in top_items])
        recommend_list_items_id_list.append(top_items)
    return recommend_list, recommend_list_items_id_list

In [48]:
user_id = test_seen_df["user_id"].to_dict()

users_list =[]
for i in range(len(user_id)):
    users_list.append(user_id[i])

In [49]:
recommend_list, recommend_list_items_id_list = sample_recommendation(model, users_list, user_id, top=50)  # test_seen_daf['user_id'].tolist()

## Save Prediction

In [50]:
import csv
with open("predict.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerow(["user_id", "course_id"])
    for i in range(len(recommend_list)):
        recommend = " ".join(recommend_list[i])
        writer.writerow([user_id[i], recommend])

## MAP 50 evaluate

In [30]:
import ml_metrics
predictions = recommend_list_items_id_list
answers = []
for user in users_list:
    prediction = recommend_list_items_id_list
    answer = []

    user_index = user_id_mapping[user]
    for item_index in range(valid_user_item_matrix.shape[1]):
        if valid_user_item_matrix[user_index][item_index] == 1:
            answer.append(item_index)
    
    answers.append(answer)
        

a = ml_metrics.mapk(predictions, answers, k = 50)
a

0.013876812922004683

In [ ]:
predictions

In [ ]:
print(predictions)

In [ ]:
recommend_list_items_id_list